In [ ]:
import torchvision.models as models
import torch
import torch.optim as optim
import torchvision.datasets as datasets
import torchvision.transforms as transforms 
from torch import nn
from torch.optim import lr_scheduler
from torch.utils.data import DataLoader,TensorDataset
import copy
import time
from torch.autograd import Variable
from keras.datasets import cifar10
import foolbox as fb
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np
import os
from utils.train import train_model,set_dataset,set_architecture

In [ ]:
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ["CUDA_VISIBLE_DEVICES"] = "3"

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
train_data_origin, test_data_origin, class_num = set_dataset("cifar10")

In [ ]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

In [ ]:
batch_size = 128

test_loader = torch.utils.data.DataLoader(
                dataset=test_data_origin,
                batch_size=batch_size,
                shuffle=True)

In [ ]:
(X_train, y_train), (X_test1, y_test1) = cifar10.load_data()
X_train=X_train.reshape(50000,3,32,32)
y_train = y_train.reshape(1,50000)
X_train=X_train/255
X_train=X_train.astype(np.float32)

In [ ]:
if not os.path.exists("./models/attack_model/TLPA"):
    os.makedirs("./models/attack_model/TLPA")
for percentage in range(1,10,2):
    model = set_architecture('Resnet18',device, 10)
    model = model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer_ft = optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=5e-4)
    exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)


    #adversarial attack generation
    for epochs in range(10):
        train_data_X=[]
        train_data_Y=[]
        bounds = (0,255)
        fmodel = fb.PyTorchModel(model, bounds=bounds)

        for k in range(int(7.8125*(percentage+1))):
            X_test=torch.from_numpy(X_train[0+32*k:32+32*k]).float().to(device)
            y_test=torch.from_numpy(y_train[0][0+32*k:32+32*k]).to(device)
            attack = fb.attacks.FGSM()
            raw, clipped, is_adv = attack(fmodel,X_test,y_test, epsilons=0.001)
            for i in range(32):
                plt.figure(num=None, figsize=(4,3), dpi=150)
                plt.figure(figsize = (0.032,0.032))
                train_data_X.append(np.array(clipped[i].cpu()))
                train_data_Y.append(y_test[i].cpu())
        X=np.array(train_data_X)
        Y=np.array(train_data_Y)
        X=torch.from_numpy(X)
        Y=torch.from_numpy(Y)
        train_data = torch.utils.data.ConcatDataset([TensorDataset(X,Y), train_data_origin])
        train_loader = DataLoader(train_data,batch_size=32, shuffle=False)
        model, acc=train_model(model, criterion, optimizer_ft, exp_lr_scheduler,device,train_loader, test_loader, num_epochs=1)
        
    torch.save(model,  './models/attack_model/TLPA/TLPA_'+str(percentage)+'.pt')
    acc_file=open('./models/attack_model/TLPA/TLPA_'+str(percentage)+'.txt','w')
    acc_file.write(str(acc))
    acc_file.close()